In [1]:
#读取h5 文件
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import h5py
import sys
from numba import jit
import seaborn as sns
from mylab.ana.miniscope.Mquality import *
from mylab.ana.miniscope.Mgraph import *
from mylab.process.miniscope.Mfunctions import savedict2mat
import scipy.io as scio
import networkx as nx

C:\Users\Sabri\Anaconda3\envs\mylab\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)


In [2]:
f = scio.loadmat(r"C:\Users\Sabri\Anaconda3\envs\mylab\Lib\site-packages\mylab\developing\graph\dpCA1_pre_post_all_cells_excl_freeze.mat")
keys = [i for i in f.keys() if r"__" not in i]
keys

['ShockActivated',
 'ShockInhibited',
 'ShockNoResponse',
 'nCells',
 'postA',
 'postB',
 'preA',
 'preB']

In [3]:
ShockActivated = pd.DataFrame([(int(np.floor(i/10000)),i%10000) for i in list(f['ShockActivated'].ravel())],columns=["mouse_id","cell_id"])
ShockActivated["type"] = "SA"
ShockInhibited = pd.DataFrame([(int(np.floor(i/10000)),i%10000) for i in list(f['ShockInhibited'].ravel())],columns=["mouse_id","cell_id"])
ShockInhibited["type"] = "SI"
ShockNoResponse =pd.DataFrame([(int(np.floor(i/10000)),i%10000) for i in  list(f['ShockNoResponse'].ravel())],columns=["mouse_id","cell_id"])
ShockNoResponse["type"] = "SN"

# neuronal infomation
all_neuron_type = pd.concat([ShockActivated,ShockInhibited,ShockNoResponse])
all_neuron_type= all_neuron_type.sort_values(by=["mouse_id","cell_id"],ignore_index=True)
all_neuron_type

,mouse_id,cell_id,type
0,1,1,SI
1,1,2,SN
2,1,3,SI
3,1,4,SA
4,1,5,SI
...,...,...,...
2329,9,158,SA
2330,9,159,SA
2331,9,160,SN
2332,9,161,SN


In [4]:

results = []
thresh = 0.3
for mouseid in [1,2,3,4,5,6,7,8,9]:
    neuron_id = all_neuron_type[all_neuron_type["mouse_id"]==mouseid].reset_index(drop=True)
    for session in ["preA","preB","postA","postB"]:
        # firing in session
        arr = f[session].T[all_neuron_type["mouse_id"]==mouseid]  
        
        # correlated pair ratio
        cpr = compute_correlated_pair_ratio(arr,threshold=thresh)
        cpr = pd.DataFrame({
            "id_index":list(cpr.keys()),
            "cpr":list(cpr.values())
        })
        
        # clustering coefficient
        cc = comput_clustering_coefficient_by_session(arr,threshold=thresh)
        cc = pd.DataFrame(cc).reset_index()
        cc.columns=["id_index","cc"]
        
        #component probability
        cps = {}
        for type in set(neuron_id["type"]):
            neuron_index = neuron_id[neuron_id["type"]==type].index
            new_arr = arr[neuron_index]
            cp = comput_component_probability_by_session(new_arr,threshold=thresh,comp_size=6)
            cps[type]=cp
        cps = pd.DataFrame({
            "type":list(cps.keys()),
            "cp":list(cps.values())
        })
        
        # connections for partner-cell
        df = pd.DataFrame(np.corrcoef(arr),columns=np.arange(0,arr.shape[0]))
        connections = corrdf_to_dict(df,cutoff=thresh)
        connections = pd.DataFrame({
            "id_index_1":np.array(list(connections.keys()))[:,0],
            "id_index_2":np.array(list(connections.keys()))[:,1],
            "component_probability":list(connections.values())
        })
        
        results.append({
            "mouse_id":mouseid,
            "session":session,
            "thresh":thresh,
            "connections":connections,
            "neuron_id":neuron_id,
            "correlated_pair_ratio":cpr,
            "clusterint_coeeficient":cc,
            "component_probability":cps
            
        })
        
        
    


C:\Users\Sabri\Anaconda3\envs\mylab\lib\site-packages\numpy\lib\function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\Sabri\Anaconda3\envs\mylab\lib\site-packages\numpy\lib\function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
C:\Users\Sabri\Anaconda3\envs\mylab\lib\site-packages\mylab\ana\miniscope\Mgraph.py:22: RuntimeWarning: invalid value encountered in greater
  rows2,cols2 = np.where(arr_cor>threshold)


In [5]:
for result in results:
    savedict2mat(r"C:\Users\Sabri\Anaconda3\envs\mylab\Lib\site-packages\mylab\developing\graph\dpCA1_result_%s_%s.mat"%(result["mouse_id"],result["session"]),result)

df is a dict
saved C:\Users\Sabri\Anaconda3\envs\mylab\Lib\site-packages\mylab\developing\graph\dpCA1_result_1_preA.mat
df is a dict
saved C:\Users\Sabri\Anaconda3\envs\mylab\Lib\site-packages\mylab\developing\graph\dpCA1_result_1_preB.mat
df is a dict
saved C:\Users\Sabri\Anaconda3\envs\mylab\Lib\site-packages\mylab\developing\graph\dpCA1_result_1_postA.mat
df is a dict
saved C:\Users\Sabri\Anaconda3\envs\mylab\Lib\site-packages\mylab\developing\graph\dpCA1_result_1_postB.mat
df is a dict
saved C:\Users\Sabri\Anaconda3\envs\mylab\Lib\site-packages\mylab\developing\graph\dpCA1_result_2_preA.mat
df is a dict
saved C:\Users\Sabri\Anaconda3\envs\mylab\Lib\site-packages\mylab\developing\graph\dpCA1_result_2_preB.mat
df is a dict
saved C:\Users\Sabri\Anaconda3\envs\mylab\Lib\site-packages\mylab\developing\graph\dpCA1_result_2_postA.mat
df is a dict
saved C:\Users\Sabri\Anaconda3\envs\mylab\Lib\site-packages\mylab\developing\graph\dpCA1_result_2_postB.mat
df is a dict
saved C:\Users\Sabri\An

# definition

### correlated pair ratio
**Significantly correlated pairs of neurons** were defined based on
the correlation coefficients of Ca2+ transient events within one
second time bins during context exploration.

**were computed for each neuron**

$$The\ correlated\ pair\ ratio =\frac{\#pairs}{\#total\ cells\ in\ FOV}$$


`An R thresholds of 0.3 was applied to this matrix to reduce bias from noise.
R thresholds at p < 0.05 from the shuffle distribution were then applied to determine the number of correlated pairs with shuffled R thresholds(Jimenez et al. 2020)`


            
        


### clustering coefficient
 the
extent to which a given neuron’s pairs were also correlated with
each other (clustering coefficient)
$$Clustering\ coefficient =\frac{\# edges\ among\ neighbors}{\# possible\ edges\ among\ neighbors} $$

`Connection density is the actual number of edges in the graph as a proportion of the total number of possible edges and is the simplest estimator of the physical cost — for example, the energy or other resource requirements — of a network. (Bullmore et al. 2009)`

**were computed for each neuron**

Total number of possible edges is: n(n-1)/2,where n is the number of nodes (neurons) in the graph.

correlated function in class NeuronalNetwork: `self.compute_connection_density()`


### component probability
A connected component is defined as a subgraph of nodes connected to each other by paths but not
connected to any other nodes. **Each neuron was assigned either a 0 or 1 indicating
whether it was a component member**, and the proportion of neurons in a component (component probability) was computed for each imaging session



correlated function in module `Mgraph.py`
`frac_nodes_in_component(g,threshold)`
**threshold** means the minimum component size.
`Due to the sparse nature of these neural graphs, we also determined whether each neuron was as member of a connected component containing at least two other neurons(Jimenez et al. 2020)`which means threshold in this paper is **3**.
$$Component\ probability =\frac{\#\ neurons\  in\  a\ component}{\#neurons\ in\  session}$$